In [13]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np

# Define the face cropping function (stub, implement as needed)
def face_cropped(img):
    # Replace with your face detection logic
    return img  # Stub for testing, you must implement proper face detection

# Create the main window
window = tk.Tk()
window.title("Face Recognition System")
window.config(background="light blue")

# Name Label and Entry
l1 = tk.Label(window, text="Name", font=("Cambria", 15))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

# Age Label and Entry
l2 = tk.Label(window, text="Age", font=("Cambria", 15))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

# Address Label and Entry
l3 = tk.Label(window, text="Address", font=("Cambria", 15))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

# Function to generate dataset
def generate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showinfo('Result', 'Please enter complete details...')
    else:
        cap = cv2.VideoCapture(0)  # Use the default camera
        if not cap.isOpened():
            print("Error: Unable to access the camera.")
            return

        id = 2
        img_id = 0
        print("Starting to collect samples. Press 'q' to quit.")

        while True:
            ret, frame = cap.read()  # Capture a frame from the camera
            if not ret or frame is None:
                print("Error: Failed to capture frame. Skipping...")
                continue

            face = face_cropped(frame)
            if face is not None:
                img_id += 1
                face = cv2.resize(face, (200, 200))
                file_name = f"C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.{str(id)}.{str(img_id)}.jpg"
                cv2.imwrite(file_name, face)
                print(f"Image {img_id} saved at {file_name}.")

                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.imshow("Cropped Face", face)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result', 'Generating dataset completed !!!')

def train_classifier():
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]  #Joins the directory path (data_dir) with each filename (f), producing the full path to each file.
    
    faces = []  #for storing images
    ids = []       #for storing ids
    
    # Loop through the image files
    for image_path in path:
        # Check if the file is a valid image file
        if image_path.endswith(".jpg") or image_path.endswith(".jpeg"):
            # Open the image and convert to grayscale
            img = Image.open(image_path).convert('L')   #simplifies face recognition by reducing complexity.
            imageNp = np.array(img, 'uint8')   #image is  converted into numpy array - unsigned 8 bit integer suitable for processing by openCV
            
            # Extract user ID from filename (assuming the format "user.1.1.jpg")
            filename = os.path.basename(image_path)
            try:
                user_id = int(filename.split(".")[1])  # Extract ID from filename (e.g., user.1.1 -> 1)
            except ValueError:
                print(f"Skipping invalid filename format: {filename}")
                continue
            
            faces.append(imageNp)
            ids.append(user_id)
    
    # Check if valid data is found
    if len(faces) == 0 or len(ids) == 0:
        print("No valid faces or IDs were found. Please check the images.")
        return

    ids = np.array(ids)

#     print(cv2.__version__)
#     print(cv2.face)

    # Train and save the classifier
    clf = cv2.face.LBPHFaceRecognizer_create()    #local binary pattern histogram - it is popular algorithm for face recognition in opencv
    clf.train(faces, ids)
    clf.write(r"C:\Users\91637\.ipynb_checkpoints\Face Recognizer\classifier.xml")
    messagebox.showinfo('Result','Training dataset completed !!!')

# Set your data directory path
data_dir = r"C:\Users\91637\.ipynb_checkpoints\Face Recognizer\data"

def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        if img is None or img.size == 0:
            print("Error: Received an empty frame.")
            return img
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            try:
                id, pred = clf.predict(gray_img[y:y+h, x:x+w])
                confidence = int(100 * (1 - pred / 300))

                if confidence > 71:
                    print(f"Predicted ID: {id}, Confidence: {confidence}")  # Debug statement
    
                    if id == 1:
                        cv2.putText(img, "Vaishu", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
                    elif id == 2:
                        cv2.putText(img, "nethaji", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
                    else:
                        cv2.putText(img, f"ID: {id}", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

#                 if confidence > 85:
#                     if id == 1:
#                         cv2.putText(img, "vaishu", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
#                     elif id == 2:
#                         cv2.putText(img, "Usha Amma", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
#                 else:
#                     cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

            except Exception as e:
                    print(f"Error during prediction: {e}")
                    cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

        return img

    # Load the classifier
    # faceCascade = cv2.CascadeClassifier(r"haarcascade_frontalface_default.xml")
    # faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascade_path)

    if faceCascade.empty():
        print("Error: Unable to load haarcascade_frontalface_default.xml.")
        exit()

    clf = cv2.face.LBPHFaceRecognizer_create()

    # Check if classifier file exists
    if not os.path.exists("classifier.xml"):
        print("Error: classifier.xml not found.")
        exit()

    clf.read("classifier.xml")

    # Open webcam
    video_capture = cv2.VideoCapture(0)  # Use index 0 for primary camera

    if not video_capture.isOpened():
        print("Error: Could not access the webcam.")
        exit()

    while True:
        ret, img = video_capture.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) == 13:  # Press Enter to exit
            break

    video_capture.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Result','Face detected !!!')



# Buttons
b1 = tk.Button(window, text="Training", font=("Cambria", 15), bg='pink', fg='red',command=train_classifier)
b1.grid(column=0, row=4)

b2 = tk.Button(window, text="Detect the face", font=("Cambria", 15), bg='pink', fg='red',command=detect_face)
b2.grid(column=1, row=4)

b3 = tk.Button(window, text="Generate Dataset", font=("Cambria", 15), bg='pink', fg='red', command=generate_dataset)
b3.grid(column=2, row=4)

# Configure window size
window.geometry("800x200")
window.mainloop()


Starting to collect samples. Press 'q' to quit.
Image 1 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.1.jpg.
Image 2 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.2.jpg.
Image 3 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.3.jpg.
Image 4 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.4.jpg.
Image 5 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.5.jpg.
Image 6 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.6.jpg.
Image 7 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.7.jpg.
Image 8 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.8.jpg.
Image 9 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.9.jpg.
Image 10 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.10.jpg.
Image 11 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.11.jpg.
Im

Image 96 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.96.jpg.
Image 97 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.97.jpg.
Image 98 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.98.jpg.
Image 99 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.99.jpg.
Image 100 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.100.jpg.
Image 101 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.101.jpg.
Image 102 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.102.jpg.
Image 103 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.103.jpg.
Image 104 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.104.jpg.
Image 105 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.105.jpg.
Image 106 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.106.jpg.
Image 107 saved at

Image 192 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.192.jpg.
Image 193 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.193.jpg.
Image 194 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.194.jpg.
Image 195 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.195.jpg.
Image 196 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.196.jpg.
Image 197 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.197.jpg.
Image 198 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.198.jpg.
Image 199 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.199.jpg.
Image 200 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.200.jpg.
Image 201 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.201.jpg.
Image 202 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.202.jpg.
Image 203 

Image 285 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.285.jpg.
Image 286 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.286.jpg.
Image 287 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.287.jpg.
Image 288 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.288.jpg.
Image 289 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.289.jpg.
Image 290 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.290.jpg.
Image 291 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.291.jpg.
Image 292 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.292.jpg.
Image 293 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.293.jpg.
Image 294 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.294.jpg.
Image 295 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.295.jpg.
Image 296 

Image 380 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.380.jpg.
Image 381 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.381.jpg.
Image 382 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.382.jpg.
Image 383 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.383.jpg.
Image 384 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.384.jpg.
Image 385 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.385.jpg.
Image 386 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.386.jpg.
Image 387 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.387.jpg.
Image 388 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.388.jpg.
Image 389 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.389.jpg.
Image 390 saved at C:/Users/91637/.ipynb_checkpoints/Face Recognizer/data/user.2.390.jpg.
Image 391 